## Figure 5: Climate baseline

In [ ]:
# spatial libraries
from shapely.geometry import Polygon
import rioxarray as rioxr
import geopandas as gpd
import xarray as xr
import xesmf as xe

import pandas as pd
import numpy as np
import cftime
import os

# plotting libraries
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

# colors and main folder
cl = px.colors.qualitative.D3
os.chdir('/home/rooda/Dropbox/Patagonia/')

In [ ]:
# Basins
basins = gpd.read_file("GIS South/Basins_Patagonia_all.shp")
basins = basins.set_index("ID")

In [ ]:
# Climate baseline (1980-2019)
period = slice("1980-01-01", "2019-12-31")

pp_pmet   = xr.open_dataset("Data/Precipitation/PP_PMET_1980_2020m.nc").sel(time = period).rename({'latitude':'lat', 'longitude':'lon'}).drop("crs")
pp_era5   = xr.open_dataset("Data/Precipitation/PP_ERA5_1959_2021m.nc").sel(time = period)
pp_cr2met = xr.open_dataset("Data/Precipitation/PP_CR2MET_1979_2020m.nc").sel(time = period)

t2m_pmet   = xr.open_dataset("Data/Temperature/Tavg_PMET_1980_2020m.nc").sel(time = period).rename({'latitude':'lat', 'longitude':'lon'})
t2m_era5   = xr.open_dataset("Data/Temperature/Tavg_ERA5_1959_2021m.nc").sel(time = period)
t2m_cr2met = xr.open_dataset("Data/Temperature/Tavg_CR2MET_1979_2020m.nc").sel(time = period)

In [ ]:
# Average annual value for selected period
pp_pmet    = pp_pmet.resample(time='1Y').sum().mean(dim="time")
pp_era5    = pp_era5.resample(time='1Y').sum().mean(dim="time")
pp_cr2met  = pp_cr2met.resample(time='1Y').sum().mean(dim="time")

t2m_pmet   = t2m_pmet.resample(time='1Y').mean().mean(dim="time")
t2m_era5   = t2m_era5.resample(time='1Y').mean().mean(dim="time")
t2m_cr2met = t2m_cr2met.resample(time='1Y').mean().mean(dim="time")

In [ ]:
# Correction of zero values
pp_pmet = pp_pmet.where(pp_pmet > 0)
pp_cr2met = pp_cr2met.where(pp_cr2met > 0)

In [ ]:
pp_pmet_avg    = xe.SpatialAverager(pp_pmet,   basins.geometry, geom_dim_name="avg")
pp_era5_avg    = xe.SpatialAverager(pp_era5,   basins.geometry, geom_dim_name="avg")
pp_cr2met_avg  = xe.SpatialAverager(pp_cr2met, basins.geometry, geom_dim_name="avg")

t2m_pmet_avg   = xe.SpatialAverager(t2m_pmet,   basins.geometry, geom_dim_name="avg")
t2m_era5_avg   = xe.SpatialAverager(t2m_era5,   basins.geometry, geom_dim_name="avg")
t2m_cr2met_avg = xe.SpatialAverager(t2m_cr2met, basins.geometry, geom_dim_name="avg")

In [ ]:
basins["PP_PMET"]   = pp_pmet_avg(pp_pmet.pp, skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["PP_ERA5"]   = pp_era5_avg(pp_era5.pp, skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["PP_CR2MET"] = pp_cr2met_avg(pp_cr2met.pp, skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["PP_ERA5d"]  = basins.PP_ERA5 - basins.PP_PMET
basins["PP_CR2METd"]= basins.PP_CR2MET - basins.PP_PMET

basins["T2M_PMET"]   = t2m_pmet_avg(t2m_pmet.t2m, skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["T2M_ERA5"]   = t2m_era5_avg(t2m_era5.t2m, skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["T2M_CR2MET"] = t2m_cr2met_avg(t2m_cr2met.t2m, skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["T2M_ERA5d"]  =  basins.T2M_ERA5 - basins.T2M_PMET
basins["T2M_CR2METd"]=  basins.T2M_CR2MET - basins.T2M_PMET

In [ ]:
geo_map = gpd.read_file("/home/rooda/Dropbox/ArcGIS/Chile/south_america.shp")
geo_map = geo_map[(geo_map.CC == "CI") | (geo_map.CC == "AR")]
geo_map = geo_map.dissolve(by='REGION')
geo_map["geometry"] = geo_map.simplify(0.01)

poly_gdf = Polygon([(-76, -55.7), (-76, -40.52), (-68.05, -40.52), (-68.05, -55.7), (-76, -55.8)])
poly_gdf = gpd.GeoDataFrame([1], geometry=[poly_gdf], crs=geo_map.crs)
geo_map = geo_map.clip(poly_gdf)

In [ ]:
row_titles = ["Annual precipitation (mm)", "Annual temperature (ºC)"]
column_titles = ["PMET", "ERA5 - PMET", "CR2MET - PMET"]

fig = make_subplots(rows=2, cols=3, horizontal_spacing = 0.01, vertical_spacing = 0.01, column_titles = column_titles, row_titles = row_titles,
                    specs=[[{"type": "scattergeo"}, {"type": "scattergeo"}, {"type": "scattergeo"}],
                           [{"type": "scattergeo"}, {"type": "scattergeo"}, {"type": "scattergeo"}]])

# Precipitation -----------------------------------------------------------------------------------------------------------
fig.add_trace(go.Choropleth(geojson = eval(geo_map['geometry'].to_json()),  locations = geo_map.index, z = geo_map['iso_num'], 
                            colorscale = ["#d5d5d5", "#d5d5d5"], showscale= False, marker_line_color ='white', marker_line_width=0.1), row=1, col=1)

fig.add_trace(go.Choropleth(geojson = eval(basins['geometry'].to_json()),  locations = basins.index, z = basins['PP_PMET'], 
                            colorscale = ["#ccebc5", "#4eb3d3", "#034b8a"], marker_line_color ='white', marker_line_width=0.1, 
                            zmin = 0, zmax = 8000, colorbar=dict(len=0.20, x=0.23, y= 0.90, title='PP (mm)', thickness=20)), row=1, col=1)

# ----
fig.add_trace(go.Choropleth(geojson = eval(geo_map['geometry'].to_json()),  locations = geo_map.index, z = geo_map['iso_num'], 
                            colorscale = ["#d5d5d5", "#d5d5d5"], showscale= False, marker_line_color ='white', marker_line_width=0.1), row=1, col=2)

fig.add_trace(go.Choropleth(geojson = eval(basins['geometry'].to_json()), locations = basins.index, z = basins['PP_ERA5d'], 
                            colorscale=["#fe7e0d","#ffe9ba","#1d78b4"], marker_line_color='white', marker_line_width=0.1, 
                            colorbar=dict(len=0.20, x=0.55, y = 0.90, title='ΔPP (mm)', thickness=20), zmid = 0, zmax = 2000, zmin = -2000), row=1, col=2)

# ----
fig.add_trace(go.Choropleth(geojson = eval(geo_map['geometry'].to_json()),  locations = geo_map.index, z = geo_map['iso_num'], 
                            colorscale = ["#d5d5d5", "#d5d5d5"], showscale= False, marker_line_color ='white', marker_line_width=0.1), row=1, col=3)

fig.add_trace(go.Choropleth(geojson = eval(basins['geometry'].to_json()), locations = basins.index, z = basins['PP_CR2METd'],
                            colorscale=["#fe7e0d","#ffe9ba","#1d78b4"], marker_line_color='white', marker_line_width=0.1, 
                            colorbar=dict(len=0.20, x=0.89, y = 0.90, title='ΔPP (mm)', thickness=20), zmid = 0, zmax = 2000, zmin = -2000), row=1, col=3)

# Temperature -----------------------------------------------------------------------------------------------------------
fig.add_trace(go.Choropleth(geojson = eval(geo_map['geometry'].to_json()),  locations = geo_map.index, z = geo_map['iso_num'], 
                            colorscale = ["#d5d5d5", "#d5d5d5"], showscale= False, marker_line_color ='white', marker_line_width=0.1), row=2, col=1)

cs = px.colors.sequential.Sunset
fig.add_trace(go.Choropleth(geojson = eval(basins['geometry'].to_json()), locations = basins.index, z = basins['T2M_PMET'], 
                            colorscale= [cs[5], cs[3],  cs[0]], marker_line_color='white', marker_line_width=0.1, 
                            zmin = -2, zmax = 8, colorbar=dict(len=0.2, x=0.23, y= 0.39, title='T2M (ºC)', thickness=20)), row=2, col=1)

# ----
fig.add_trace(go.Choropleth(geojson = eval(geo_map['geometry'].to_json()),  locations = geo_map.index, z = geo_map['iso_num'], 
                            colorscale = ["#d5d5d5", "#d5d5d5"], showscale= False, marker_line_color ='white', marker_line_width=0.1), row=2, col=2)

fig.add_trace(go.Choropleth(geojson = eval(basins['geometry'].to_json()), locations = basins.index, z = basins['T2M_ERA5d'], 
                            colorscale=["#1d78b4", "#ffe9ba","#fe7e0d"], marker_line_color='white', marker_line_width=0.1, 
                            colorbar=dict(len=0.2, x=0.55, y = 0.39, title='ΔT2M (ºC)', thickness=20), zmid = 0, zmax = 2, zmin = -2), row=2, col=2)

# ----
fig.add_trace(go.Choropleth(geojson = eval(geo_map['geometry'].to_json()),  locations = geo_map.index, z = geo_map['iso_num'], 
                            colorscale = ["#d5d5d5", "#d5d5d5"], showscale= False, marker_line_color ='white', marker_line_width=0.1), row=2, col=3)

fig.add_trace(go.Choropleth(geojson = eval(basins['geometry'].to_json()), locations = basins.index, z = basins['T2M_CR2METd'],
                            colorscale=["#1d78b4", "#ffe9ba","#fe7e0d"],  marker_line_color='white', marker_line_width=0.1, 
                            colorbar=dict(len=0.2, x=0.89, y = 0.39, title='ΔT2M (ºC)', thickness=20), zmid = 0, zmax = 2, zmin = -2), row=2, col=3)

# Layout for all -------------------------------------------------------------------------------------------------------------
fig.update_geos(showframe = True, framewidth = 1, framecolor = "rgba(0,0,0,0.5)", lonaxis_range=[-76, -68], lataxis_range=[-55.8, -40.5], 
                bgcolor = "#f9f9f9", showland = False, showcoastlines = False, showlakes = False)

fig.update_layout(autosize = False, width = 900, height = 1150, margin = dict(l=7, r=15, b=7, t=30))
fig.write_image("/home/rooda/Dropbox/Patagonia/MS2 Results/Figure_5_climate.png", scale=4)
fig.show()